<a href="https://colab.research.google.com/github/0WhitherGgr0/ComputacionVisual_G4/blob/main/Tarea%20de%20la%20semana%209/%20CV_Semana09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grupo 4

Integrantes:

*   Calderon Matias Diego Alonso
*   Calderon Zuñiga Rodrigo Joaquin
*   Chochoca Yarleque Bruno Omar
*   Levis Escobedo Eduardo Valentín
*   Chaco Flores Jose Luis

In [ ]:
# Instalar CUDA
!apt-get update
!apt-get install -y cuda

# Verificar la instalación de nvcc
!nvcc --version

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,400 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,255 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates

In [ ]:
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp3mh2e5wg".


**1. Suma de dos vectores de dimensión 1 000 000 en CPU:**



In [ ]:
%%writefile cpu.cpp
#include <iostream>
#include <vector>
#include <chrono>

using namespace std;

int main() {
    const int size = 1000000;
    vector<float> a(size, 1.0f); // Inicializa el vector con 1.0
    vector<float> b(size, 2.0f); // Inicializa el vector con 2.0
    vector<float> c(size, 0.0f); // Vector para almacenar el resultado

    auto start = chrono::high_resolution_clock::now();

    // Suma de vectores en CPU
    for (int i = 0; i < size; ++i) {
        c[i] = a[i] + b[i];
    }

    auto end = chrono::high_resolution_clock::now();
    chrono::duration<float, milli> duration = end - start;

    cout << "Tiempo de ejecución en CPU: " << duration.count() << " ms" << endl;

    return 0;
}


Overwriting cpu.cpp


In [ ]:
!g++ cpu.cpp && ./a.out

Tiempo de ejecución en CPU: 7.25851 ms


**2. Suma de dos vectores de dimensión 1 000 000 en GPU:**

In [ ]:
%%writefile gpu.cu
#include <cuda_runtime.h>
#include <iostream>
#include <chrono>

using namespace std;

__global__ void vectorAdd(const float* a, const float* b, float* c, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
    }
}

int main() {
    const int n = 1000000;
    const int bytes = n * sizeof(float);

    float *h_a = new float[n];
    float *h_b = new float[n];
    float *h_c = new float[n];

    // Inicializa los vectores en la CPU
    for (int i = 0; i < n; ++i) {
        h_a[i] = 1.0f;
        h_b[i] = 2.0f;
    }

    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);

    auto start = chrono::high_resolution_clock::now();

    cudaMemcpy(d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, bytes, cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;

    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);
    cudaDeviceSynchronize();

    cudaMemcpy(h_c, d_c, bytes, cudaMemcpyDeviceToHost);

    auto end = chrono::high_resolution_clock::now();
    chrono::duration<float, milli> duration = end - start;

    cout << "Tiempo de ejecución en GPU: " << duration.count() << " ms" << endl;

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    delete[] h_a;
    delete[] h_b;
    delete[] h_c;

    return 0;
}

Overwriting gpu.cu


In [ ]:
!nvcc gpu.cu -o gpu_executable && ./gpu_executable

Tiempo de ejecución en GPU: 0.04121 ms
